In [3]:
from LandBase import *

# 아파트매매정보를 DB에 저장 (출처: 국토교통부)
def deal_history_into_db(p_con, p_deal_ym, p_sgg_cd, p_ignore_flag=True) :
    table_name = 'deal_history'
    exist_flag = False
    
    # 기존 데이터를 무시하지 말라(p_ignore_flag==False)는 옵션이 주어지면
    # 같은 데이터가 이미 있는지 검사해서 있으면 함수를 종료함
    if (p_ignore_flag==False) :
        sql = 'select 1 as cnt from '+table_name +' where sgg_cd = \''+p_sgg_cd+'\' and deal_ym = \'' + p_deal_ym + '\' limit 1'
        exist_flag = check_data_exists(p_con, sql)
        if (exist_flag) :
            return 

    # Open API 호출
    url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade' \
    + '?LAWD_CD={sgg_cd}&DEAL_YMD={deal_ym}&serviceKey={key}'
    url = url.format(sgg_cd=p_sgg_cd, deal_ym=p_deal_ym, key=data_go_kr_apikey)

    response = requests.get(url)

    # xml 형식으로 반환된 Open API 결과를 DataFrame으로 변환
    df_deal = xml2df(response.content, p_element_name='items', p_level=2) 
    if df_deal.empty : return
    df_deal.drop(['년','월'], axis=1, inplace=True)
    df_deal.insert(0, 'deal_ym', p_deal_ym)
    df_deal['거래금액'] = df_deal['거래금액'].str.replace(',', '')
    df_deal['법정동'] = df_deal['법정동'].str.strip()
    df_deal['아파트'] = df_deal['아파트'].str.strip()
    df_deal.rename(columns={'거래금액':'deal_amount', \
                            '건축년도':'built_yyyy', \
                            '법정동':'law_dong_nm', \
                            '아파트':'apt_nm', \
                            '일':'deal_dd', \
                            '전용면적':'priv_area', \
                            '지번':'post_no', \
                            '지역코드':'sgg_cd', \
                            '층':'floor'}, \
                inplace=True)

    # DB 처리
    con.execute('delete from ' + table_name + ' where sgg_cd = \'' + p_sgg_cd + '\' and deal_ym = \'' + p_deal_ym + '\'')
    df_deal.to_sql(name=table_name, con=p_con,  if_exists='append', index=False, chunksize=1000)
    sql = 'update deal_history h ' \
    + 'set h.lawd_cd = ' \
    + '    (select a.lawd_cd ' \
    + '       from adm_law_map a ' \
    + '      where a.lawd_cd like concat(h.sgg_cd, \'%\') ' \
    + '        and a.lawd_nm = trim(substring_index(h.law_dong_nm, \' \', -1)) ' \
    + '      order by a.from_dt desc ' \
    + '      limit 1 ) ' \
    + ' where h.lawd_cd is null' \
    + '   and h.sgg_cd = \'' + p_sgg_cd + '\'' \
    + '   and h.deal_ym = \'' + p_deal_ym + '\''
    con.execute(sql)
    con.execute('commit')

# 아파트전월세정보를 DB에 저장 (출처: 국토교통부)
def rent_history_into_db(p_con, p_deal_ym, p_sgg_cd, p_ignore_flag=True) :
    table_name = 'rent_history'
    exist_flag = False

    # 기존 데이터를 무시하지 말라(p_ignore_flag==False)는 옵션이 주어지면
    # 같은 데이터가 이미 있는지 검사해서 있으면 함수를 종료함
    if (p_ignore_flag==False) :
        sql = 'select 1 as cnt from '+table_name +' where sgg_cd = \''+p_sgg_cd+'\' and deal_ym = \'' + p_deal_ym + '\' limit 1'
        exist_flag = check_data_exists(p_con, sql)
        if (exist_flag) :
            return

    # Open API 호출
    url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent' \
    + '?LAWD_CD={sgg_cd}&DEAL_YMD={deal_ym}&serviceKey={key}'
    url = url.format(sgg_cd=p_sgg_cd, deal_ym=p_deal_ym, key=data_go_kr_apikey)
    response = requests.get(url)

    # xml 형식으로 반환된 Open API 결과를 DataFrame으로 변환
    df_rent = xml2df(response.content, p_element_name='items', p_level=2)
    if df_rent.empty : return
    df_rent.drop(['년','월','건축년도'], axis=1, inplace=True)
    df_rent.insert(0, 'deal_ym', p_deal_ym)
    df_rent['보증금액'] = df_rent['보증금액'].str.replace(',', '')
    df_rent['월세금액'] = df_rent['월세금액'].str.replace(',', '')
    df_rent['법정동'] = df_rent['법정동'].str.strip()
    df_rent['아파트'] = df_rent['아파트'].str.strip()
    df_rent.rename(columns={'보증금액':'deposit_amount', \
                            '건축년도':'built_yyyy', \
                            '월세금액':'m_rent_amount', \
                            '법정동':'law_dong_nm', \
                            '아파트':'apt_nm', \
                            '일':'deal_dd', \
                            '전용면적':'priv_area', \
                            '지번':'post_no', \
                            '지역코드':'sgg_cd', \
                            '층':'floor'}, \
                inplace=True)

    # DB 처리
    con.execute('delete from ' + table_name + ' where sgg_cd = \'' + p_sgg_cd + '\' and deal_ym = \'' + p_deal_ym + '\'')
    df_rent.to_sql(name=table_name, con=p_con,  if_exists='append', index=False, chunksize=1000)
    sql = 'update rent_history h ' \
    + 'set h.lawd_cd = ' \
    + '    (select a.lawd_cd ' \
    + '       from adm_law_map a ' \
    + '      where a.lawd_cd like concat(h.sgg_cd, \'%\') ' \
    + '        and a.lawd_nm = trim(substring_index(h.law_dong_nm, \' \', -1)) ' \
    + '      order by a.from_dt desc ' \
    + '      limit 1 ) ' \
    + ' where h.lawd_cd is null' \
    + '   and h.sgg_cd = \'' + p_sgg_cd + '\'' \
    + '   and h.deal_ym = \'' + p_deal_ym + '\''
    con.execute(sql)
    con.execute('commit')

In [4]:
# DB Connection Open
con = get_db_connection()

# 아파트매매정보와 전월세 정보를 DB에 저장
# 거래년월과 시군구코드(법정동코드인 lawd_cd의 앞 5글자)를 인자로 넘김
# 테이블: deal_history
ym_list = get_ym_list('201101', '201105')
# 법정동 코드 (10자리) : 실제주소. 시도(2)+시군구(3)+읍면동(3)+리(2)
sql = 'select distinct substr(lawd_cd, 1, 5) as sgg_cd, sgg_nm'\
+ ' from adm_law_map'\
+ ' where lawd_cd like \'%00000\' and sgg_nm != \'\' and lawd_nm != \'\''\
+ ' and sgg_nm not like \'%군\' and sgg_nm not like \'%균\' order by lawd_cd'
sgg_list = con.execute(sql).fetchall()
for deal_ym in ym_list :
    print(deal_ym)
    for sgg_cd in sgg_list :
        deal_history_into_db (p_con=con, p_deal_ym=deal_ym, p_sgg_cd=sgg_cd[0], p_ignore_flag=False)
        rent_history_into_db (p_con=con, p_deal_ym=deal_ym, p_sgg_cd=sgg_cd[0], p_ignore_flag=False)

# DB Connection Close
disconnect_db(con)

print('Program Finised')

201101
201102
201103
201104
201105
Program Finised


In [5]:
# DB Connection Open
con = get_db_connection()

deal_history_into_db (p_con=con, p_deal_ym='201709', p_sgg_cd='41195', p_ignore_flag=False)

disconnect_db(con)

print('Program Finised')

http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=41195&DEAL_YMD=201709&serviceKey=CHQiZX6GcYWPyDjJI5LUcCGsxWdv59igYvsKyWKSWC7qv0CkaSbn95Tf3li1rk4p4Umffrf%2FvJ%2B%2F0SrnEy9x7g%3D%3D
Program Finised
